In [1]:
# !python3 -m pip install -r requirements.txt --user

In [1]:
# Library Imports

import gc
import math
import json
import time
import uuid
import traceback
import warnings
import urllib.parse
import pandas as pd
import numpy as np
import connectorx as cx
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import Integer,Float
import psycopg2.extras as extras
warnings.filterwarnings("ignore")

In [2]:
# DATABASE PARAMETER
database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'
connetion = psycopg2.connect(database = database, user = user ,password = password, host = host,port = port)

In [3]:
# GLOBAL VARIABLE PARAMETERs
schema_name = 'tm_new_data'
volume_table  = 'tm_new_data.nys_hour_8_volume_240626'
connected_table = 'tm_new_data.ny_connected_osm_master_231211'
geh_table = 'nys_hour_8_volume_240626_adjacent_geh_check'

In [4]:
# GET BASE DATA FOR AVAILABLE VOLUME

# sql_command = f"select * from {volume_table}"
# All_Data_df = pd.read_sql(sql_command, connetion)

sql_command = f"""select distinct isg_osm_id,next_connected_count,source,highway_id,volume,single_next_ids,
                    multi_next_ids,single_prev_ids,multi_prev_ids  from
                    (select A.*,B.* from 
                        (select * from {volume_table} where volume > 0 and next_pcc = 1) A
                        ,{connected_table} B
                    where A.isg_osm_id = B.current_id 
                    )foo"""

# GET PROCESS LIST
All_Data_df = pd.read_sql(sql_command, connetion)
Updated_Available_LST = np.unique(list(set(All_Data_df.to_dict(orient='list')['isg_osm_id'])))

In [5]:
# Remove osm from list that has adjacent next/prev volume available
remove_from_next_geh_df = pd.read_sql(f'''select distinct isg_osm_id as removal_ids_next_geh from {schema_name}.{geh_table}
                                        where conn_tag in('single','multi_next')''',connetion)

remove_from_next_geh_list = remove_from_next_geh_df['removal_ids_next_geh'].unique()

next_geh_process_list = list(set(Updated_Available_LST)-set(remove_from_next_geh_list))

In [6]:
len_Updated_Available_LST = len(Updated_Available_LST)
print("Available Volume :",len_Updated_Available_LST)

print("remove_from_next_geh_list :",len(remove_from_next_geh_list)) 

len_next_geh_process_list = len(next_geh_process_list)
print("next_geh_process_list :",len_next_geh_process_list)


Available Volume : 25872
remove_from_next_geh_list : 902
next_geh_process_list : 24970


In [7]:
def geh_check(current_vol,next_vol):
    return round(math.sqrt(2*((current_vol-next_vol)*(current_vol-next_vol))/(current_vol+next_vol)),2)

In [8]:
Missing_IDS = []
error_dict = {}
iteration='Original'

augmented_df = pd.DataFrame({'isg_osm_id':[],'conn_link_array':[],'connected_count':[],
                             'source':[], 'conn_src_array':[],
                             'volume':[],'total_conn_vol':[],'conn_vol_array':[],
                             'conn_tag':[],'iteration':[],'geh':[]},dtype='int64')
# augmented_df = pd.DataFrame(dtype = 'int64')
start = time.time()


# NEXT_GEH_CALCULATION
counter = 0  
for search_osm_id in next_geh_process_list: 

    try:
        # Extract Current Search IDS Data
        cur_df = All_Data_df.loc[All_Data_df['isg_osm_id'] == search_osm_id].reset_index(drop = True)
      

        ############################## FOR SINGLE ( FORWARD AND BACKWARD )
        
        single_next_ids_lst = cur_df['single_next_ids'].iloc[0]
        cur_volume = cur_df['volume'].iloc[0]
        connected_count = cur_df['next_connected_count'].iloc[0]
        source = cur_df['source'].iloc[0]
        
        common_elements = list(set(single_next_ids_lst) & set(Updated_Available_LST))
        #print(len(common_elements))

        if len(common_elements) > 0:
            #print('single')
            
            for item in single_next_ids_lst:
                if item in common_elements:
                    conncetd_id = item
                    break
                    
            next_df = All_Data_df.loc[All_Data_df['isg_osm_id'] == conncetd_id].reset_index(drop = True)
            conn_src_array =  next_df[['isg_osm_id','source']].drop_duplicates()['source'].to_list()

            next_volume = next_df['volume'].iloc[0]
            geh=geh_check(cur_volume,next_volume)  
            single_level_df = pd.DataFrame()
            single_level_df['isg_osm_id'] = [int(search_osm_id)]
            single_level_df['conn_link_array'] = [[int(conncetd_id)]]
            single_level_df['connected_count'] = [int(connected_count)]
            single_level_df['source'] = [source]
            single_level_df['conn_src_array'] = [conn_src_array]
            single_level_df['volume'] = [int(cur_volume)]
            single_level_df['total_conn_vol'] = [int(next_volume)]
            single_level_df['conn_vol_array'] = [[int(next_volume)]]
            single_level_df['conn_tag'] = ['single']
            single_level_df['iteration'] = iteration
            single_level_df['geh'] = [float(geh)]
            augmented_df=pd.concat([augmented_df, single_level_df], ignore_index=True)
        
        ######################## FOR MULTI NEXT

        if len(common_elements) == 0:
            #print("next")
        
            multi_next_ids_lst = cur_df['multi_next_ids'].iloc[0]
            multi_next_data=All_Data_df.loc[All_Data_df['isg_osm_id'].isin(multi_next_ids_lst)].reset_index(drop = True)
            #print(len(multi_next_data),len(multi_next_ids_lst))
            conn_src_array =  multi_next_data[['isg_osm_id','source']].drop_duplicates()['source'].to_list()

            if len(multi_next_data) == len(multi_next_ids_lst) and len(multi_next_ids_lst) > 1 :
                
                next_volume = multi_next_data['volume'].sum().item()
#                 print(type(next_volume),': ',next_volume)
                geh=geh_check(cur_volume,next_volume)
                multi_next_df = pd.DataFrame()
                multi_next_df['isg_osm_id'] = [int(search_osm_id)]
                multi_next_df['conn_link_array'] = [multi_next_ids_lst]
                multi_next_df['connected_count'] = [int(connected_count)]
                multi_next_df['source'] = [source]
                multi_next_df['conn_src_array'] = [conn_src_array]
                multi_next_df['volume'] = [int(cur_volume)]
                multi_next_df['total_conn_vol'] = [int(next_volume)]
                multi_next_df['conn_vol_array'] = [multi_next_data['volume'].unique().tolist()]
                multi_next_df['conn_tag'] = ['multi_next']
                multi_next_df['iteration'] = iteration
                multi_next_df['geh'] = [float(geh)]
#                 print(multi_next_df)
                augmented_df=pd.concat([augmented_df, multi_next_df], ignore_index=True)
    
            if len(multi_next_data) < len(multi_next_ids_lst) and len(multi_next_ids_lst) > 1 and len(multi_next_data)>0:
            
                next_volume = multi_next_data['volume'].sum().item()
#                 print(type(next_volume),': ',next_volume)
                if next_volume > cur_volume:
                    geh=geh_check(cur_volume,next_volume)
                    multi_next_df = pd.DataFrame()
                    multi_next_df['isg_osm_id'] = [int(search_osm_id)]
                    multi_next_df['conn_link_array'] = [multi_next_data['isg_osm_id'].unique().tolist()]
                    multi_next_df['connected_count'] = [int(connected_count)]
                    multi_next_df['source'] = [source]
                    multi_next_df['conn_src_array'] = [conn_src_array]
                    multi_next_df['volume'] = [int(cur_volume)]
                    multi_next_df['total_conn_vol'] = [int(next_volume)]
                    multi_next_df['conn_vol_array'] = [multi_next_data['volume'].unique().tolist()]
                    multi_next_df['conn_tag'] = ['multi_next']
                    multi_next_df['iteration'] = iteration
                    multi_next_df['geh'] = [float(geh)]
    #                 print(multi_next_df)
                    augmented_df=pd.concat([augmented_df, multi_next_df], ignore_index=True)
        
        counter = counter + 1
        print("Processed ID :",counter)
        
    except:
        error_dict[str(search_osm_id)] = str(traceback.format_exc())
        Missing_IDS.append(search_osm_id)
        print("Some ID Missed")
        continue

end = time.time()

Processed ID : 1
Processed ID : 2
Processed ID : 3
Processed ID : 4
Processed ID : 5
Processed ID : 6
Processed ID : 7
Processed ID : 8
Processed ID : 9
Processed ID : 10
Processed ID : 11
Processed ID : 12
Processed ID : 13
Processed ID : 14
Processed ID : 15
Processed ID : 16
Processed ID : 17
Processed ID : 18
Processed ID : 19
Processed ID : 20
Processed ID : 21
Processed ID : 22
Processed ID : 23
Processed ID : 24
Processed ID : 25
Processed ID : 26
Processed ID : 27
Processed ID : 28
Processed ID : 29
Processed ID : 30
Processed ID : 31
Processed ID : 32
Processed ID : 33
Processed ID : 34
Processed ID : 35
Processed ID : 36
Processed ID : 37
Processed ID : 38
Processed ID : 39
Processed ID : 40
Processed ID : 41
Processed ID : 42
Processed ID : 43
Processed ID : 44
Processed ID : 45
Processed ID : 46
Processed ID : 47
Processed ID : 48
Processed ID : 49
Processed ID : 50
Processed ID : 51
Processed ID : 52
Processed ID : 53
Processed ID : 54
Processed ID : 55
Processed ID : 56
P

Processed ID : 11151
Processed ID : 11152
Processed ID : 11153
Processed ID : 11154
Processed ID : 11155
Processed ID : 11156
Processed ID : 11157
Processed ID : 11158
Processed ID : 11159
Processed ID : 11160
Processed ID : 11161
Processed ID : 11162
Processed ID : 11163
Processed ID : 11164
Processed ID : 11165
Processed ID : 11166
Processed ID : 11167
Processed ID : 11168
Processed ID : 11169
Processed ID : 11170
Processed ID : 11171
Processed ID : 11172
Processed ID : 11173
Processed ID : 11174
Processed ID : 11175
Processed ID : 11176
Processed ID : 11177
Processed ID : 11178
Processed ID : 11179
Processed ID : 11180
Processed ID : 11181
Processed ID : 11182
Processed ID : 11183
Processed ID : 11184
Processed ID : 11185
Processed ID : 11186
Processed ID : 11187
Processed ID : 11188
Processed ID : 11189
Processed ID : 11190
Processed ID : 11191
Processed ID : 11192
Processed ID : 11193
Processed ID : 11194
Processed ID : 11195
Processed ID : 11196
Processed ID : 11197
Processed ID 

Processed ID : 11547
Processed ID : 11548
Processed ID : 11549
Processed ID : 11550
Processed ID : 11551
Processed ID : 11552
Processed ID : 11553
Processed ID : 11554
Processed ID : 11555
Processed ID : 11556
Processed ID : 11557
Processed ID : 11558
Processed ID : 11559
Processed ID : 11560
Processed ID : 11561
Processed ID : 11562
Processed ID : 11563
Processed ID : 11564
Processed ID : 11565
Processed ID : 11566
Processed ID : 11567
Processed ID : 11568
Processed ID : 11569
Processed ID : 11570
Processed ID : 11571
Processed ID : 11572
Processed ID : 11573
Processed ID : 11574
Processed ID : 11575
Processed ID : 11576
Processed ID : 11577
Processed ID : 11578
Processed ID : 11579
Processed ID : 11580
Processed ID : 11581
Processed ID : 11582
Processed ID : 11583
Processed ID : 11584
Processed ID : 11585
Processed ID : 11586
Processed ID : 11587
Processed ID : 11588
Processed ID : 11589
Processed ID : 11590
Processed ID : 11591
Processed ID : 11592
Processed ID : 11593
Processed ID 

Processed ID : 11943
Processed ID : 11944
Processed ID : 11945
Processed ID : 11946
Processed ID : 11947
Processed ID : 11948
Processed ID : 11949
Processed ID : 11950
Processed ID : 11951
Processed ID : 11952
Processed ID : 11953
Processed ID : 11954
Processed ID : 11955
Processed ID : 11956
Processed ID : 11957
Processed ID : 11958
Processed ID : 11959
Processed ID : 11960
Processed ID : 11961
Processed ID : 11962
Processed ID : 11963
Processed ID : 11964
Processed ID : 11965
Processed ID : 11966
Processed ID : 11967
Processed ID : 11968
Processed ID : 11969
Processed ID : 11970
Processed ID : 11971
Processed ID : 11972
Processed ID : 11973
Processed ID : 11974
Processed ID : 11975
Processed ID : 11976
Processed ID : 11977
Processed ID : 11978
Processed ID : 11979
Processed ID : 11980
Processed ID : 11981
Processed ID : 11982
Processed ID : 11983
Processed ID : 11984
Processed ID : 11985
Processed ID : 11986
Processed ID : 11987
Processed ID : 11988
Processed ID : 11989
Processed ID 

Processed ID : 12338
Processed ID : 12339
Processed ID : 12340
Processed ID : 12341
Processed ID : 12342
Processed ID : 12343
Processed ID : 12344
Processed ID : 12345
Processed ID : 12346
Processed ID : 12347
Processed ID : 12348
Processed ID : 12349
Processed ID : 12350
Processed ID : 12351
Processed ID : 12352
Processed ID : 12353
Processed ID : 12354
Processed ID : 12355
Processed ID : 12356
Processed ID : 12357
Processed ID : 12358
Processed ID : 12359
Processed ID : 12360
Processed ID : 12361
Processed ID : 12362
Processed ID : 12363
Processed ID : 12364
Processed ID : 12365
Processed ID : 12366
Processed ID : 12367
Processed ID : 12368
Processed ID : 12369
Processed ID : 12370
Processed ID : 12371
Processed ID : 12372
Processed ID : 12373
Processed ID : 12374
Processed ID : 12375
Processed ID : 12376
Processed ID : 12377
Processed ID : 12378
Processed ID : 12379
Processed ID : 12380
Processed ID : 12381
Processed ID : 12382
Processed ID : 12383
Processed ID : 12384
Processed ID 

...

Processed ID : 41481
Processed ID : 41482
Processed ID : 41483
Processed ID : 41484
Processed ID : 41485
Processed ID : 41486
Processed ID : 41487
Processed ID : 41488
Processed ID : 41489
Processed ID : 41490
Processed ID : 41491
Processed ID : 41492
Processed ID : 41493
Processed ID : 41494
Processed ID : 41495
Processed ID : 41496
Processed ID : 41497
Processed ID : 41498
Processed ID : 41499
Processed ID : 41500
Processed ID : 41501
Processed ID : 41502
Processed ID : 41503
Processed ID : 41504
Processed ID : 41505
Processed ID : 41506
Processed ID : 41507
Processed ID : 41508
Processed ID : 41509
Processed ID : 41510
Processed ID : 41511
Processed ID : 41512
Processed ID : 41513
Processed ID : 41514
Processed ID : 41515
Processed ID : 41516
Processed ID : 41517
Processed ID : 41518
Processed ID : 41519
Processed ID : 41520
Processed ID : 41521
Processed ID : 41522
Processed ID : 41523
Processed ID : 41524
Processed ID : 41525
Processed ID : 41526
Processed ID : 41527
Processed ID 

Processed ID : 41873
Processed ID : 41874
Processed ID : 41875
Processed ID : 41876
Processed ID : 41877
Processed ID : 41878
Processed ID : 41879
Processed ID : 41880
Processed ID : 41881
Processed ID : 41882
Processed ID : 41883
Processed ID : 41884
Processed ID : 41885
Processed ID : 41886
Processed ID : 41887
Processed ID : 41888
Processed ID : 41889
Processed ID : 41890
Processed ID : 41891
Processed ID : 41892
Processed ID : 41893
Processed ID : 41894
Processed ID : 41895
Processed ID : 41896
Processed ID : 41897
Processed ID : 41898
Processed ID : 41899
Processed ID : 41900
Processed ID : 41901
Processed ID : 41902
Processed ID : 41903
Processed ID : 41904
Processed ID : 41905
Processed ID : 41906
Processed ID : 41907
Processed ID : 41908
Processed ID : 41909
Processed ID : 41910
Processed ID : 41911
Processed ID : 41912
Processed ID : 41913
Processed ID : 41914
Processed ID : 41915
Processed ID : 41916
Processed ID : 41917
Processed ID : 41918
Processed ID : 41919
Processed ID 

Processed ID : 42265
Processed ID : 42266
Processed ID : 42267
Processed ID : 42268
Processed ID : 42269
Processed ID : 42270
Processed ID : 42271
Processed ID : 42272
Processed ID : 42273
Processed ID : 42274
Processed ID : 42275
Processed ID : 42276
Processed ID : 42277
Processed ID : 42278
Processed ID : 42279
Processed ID : 42280
Processed ID : 42281
Processed ID : 42282
Processed ID : 42283
Processed ID : 42284
Processed ID : 42285
Processed ID : 42286
Processed ID : 42287
Processed ID : 42288
Processed ID : 42289
Processed ID : 42290
Processed ID : 42291
Processed ID : 42292
Processed ID : 42293
Processed ID : 42294
Processed ID : 42295
Processed ID : 42296
Processed ID : 42297
Processed ID : 42298
Processed ID : 42299
Processed ID : 42300
Processed ID : 42301
Processed ID : 42302
Processed ID : 42303
Processed ID : 42304
Processed ID : 42305
Processed ID : 42306
Processed ID : 42307
Processed ID : 42308
Processed ID : 42309
Processed ID : 42310
Processed ID : 42311
Processed ID 

Processed ID : 42658
Processed ID : 42659
Processed ID : 42660
Processed ID : 42661
Processed ID : 42662
Processed ID : 42663
Processed ID : 42664
Processed ID : 42665
Processed ID : 42666
Processed ID : 42667
Processed ID : 42668
Processed ID : 42669
Processed ID : 42670
Processed ID : 42671
Processed ID : 42672
Processed ID : 42673
Processed ID : 42674
Processed ID : 42675
Processed ID : 42676
Processed ID : 42677
Processed ID : 42678
Processed ID : 42679
Processed ID : 42680
Processed ID : 42681
Processed ID : 42682
Processed ID : 42683
Processed ID : 42684
Processed ID : 42685
Processed ID : 42686
Processed ID : 42687
Processed ID : 42688
Processed ID : 42689
Processed ID : 42690
Processed ID : 42691
Processed ID : 42692
Processed ID : 42693
Processed ID : 42694
Processed ID : 42695
Processed ID : 42696
Processed ID : 42697
Processed ID : 42698
Processed ID : 42699
Processed ID : 42700
Processed ID : 42701
Processed ID : 42702
Processed ID : 42703
Processed ID : 42704
Processed ID 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
All_Data_df[All_Data_df['isg_osm_id'].isin([5705058001,941234030002])]

Empty DataFrame
Columns: [isg_osm_id, next_connected_count, source, highway_id, volume, single_next_ids, multi_next_ids, single_prev_ids, multi_prev_ids]
Index: []

In [13]:
All_Data_df

isg_osm_id  next_connected_count    source  highway_id  volume  \
0            4350451001                     1  Original           1    2579   
1            4350451002                     1  Original           1    2877   
2            4371845002                     1  Original           1    1139   
3            5029221002                     1  Original           5    1017   
4            5564334001                     1  Original           2     217   
...                 ...                   ...       ...         ...     ...   
25867  9991051936746001                     1  Original          14     129   
25868  9991052269403010                     1  Original           7      58   
25869  9991052681624001                     1  Original          11       6   
25870  9991052843434001                     1  Original          11       3   
25871  9991054804874003                     2  Original          11     312   

                                         single_next_ids  \
0      [4350451002, 4350451003, 347697410001, 3476974...   
1      [4350451003, 347697410001, 347697409001, 86806...   
2                                           [4371845003]   
3                                         [421121606001]   
4                                        [9985642513001]   
...                                                  ...   
25867  [9991051936747003, 9991051936747002, 999105193...   
25868  [9991052269403009, 9991052269403008, 999105226...   
25869                                   [99923514932002]   
25870  [9991052843435002, 9991052843435001, 999105284...   
25871                                                 []   

                   multi_next_ids                       single_prev_ids  \
0      [24930121001, 27837482001]                                    []   
1      [24930121001, 27837482001]                          [4350451001]   
2       [23724017001, 4371845004]                                    []   
3                              []                                    []   
4                              []                                    []   
...                           ...                                   ...   
25867                          []                                    []   
25868                          []  [9991052269403011, 9991052269403012]   
25869                          []                    [9991052681645001]   
25870                          []                    [9991052843433001]   
25871                          []                                    []   

                   multi_prev_ids  
0      [24137110001, 94449377001]  
1      [24137110001, 94449377001]  
2        [5715198002, 4371845001]  
3                              []  
4                              []  
...                           ...  
25867                          []  
25868                          []  
25869                          []  
25870                          []  
25871                          []  

[25872 rows x 9 columns]

In [14]:
print("TIME TAKEN To CALCULATE GEH : ",(end - start))

TIME TAKEN To CALCULATE GEH :  114.07293105125427


In [15]:
print(len(augmented_df['isg_osm_id'].unique()))
print(len(augmented_df['isg_osm_id']))
print(len(Missing_IDS))
print(len(error_dict.keys()))

258
258
0
0


In [16]:
augmented_df[augmented_df['geh'] >5]

isg_osm_id    conn_link_array  connected_count    source  \
1    998222279148001  [999473475839001]                1  Original   
3    999668779648030  [999668779648022]                1  Original   
5    998308328048002    [9995697574006]                1  Original   
11       39487081002    [1029640287001]                1  Original   
13      270749732001     [586133525002]                1  Original   
24      898016745001      [39012951001]                1  Original   
25        5676734008     [799512727001]                1  Original   
32      420604747001     [420604747003]                1  Original   
35       46201643001     [465066386001]                1  Original   
44    99824185454001  [998314940116001]                1  Original   
46      958176047002     [570981706001]                1  Original   
48    99820641596001  [999267405091012]                1  Original   
60    99823048670001    [9985671335002]                1  Original   
63   998143479332007  [998143479332009]                1  Original   
72   999219924520005  [999219924520003]                1  Original   
89    99931775852008  [999736485360003]                1  Original   
90   999769518399017  [999769518399002]                1  Original   
96   998221608282001   [99946714392003]                1  Original   
97   998420503554001  [998420503674001]                1  Original   
100     799506983001      [46662048001]                1  Original   
102    9985711041001    [9985711041005]                1  Original   
106  998318474882001  [998992603345001]                1  Original   
111  998992603345001      [36185331001]                1  Original   
112     122738340001     [375623294001]                1  Original   
113  999586820551004   [99834961048001]                1  Original   
119   99920102455001   [99935971874021]                1  Original   
120       5706578001     [801411766001]                1  Original   
124    9985669693001    [9985669693003]                1  Original   
128    9995679138005     [109971983004]                1  Original   
129   99923477824001   [99923471879001]                1  Original   
141      20131056001      [20130288001]                1  Original   
142       5672176002     [420598310001]                1  Original   
144  999420583316001   [99946137552001]                1  Original   
147     420822552001     [420822550001]                1  Original   
151      32959795001      [32959885001]                1  Original   
158     630255622001     [947539269001]                1  Original   
159     375623293001     [158613789001]                1  Original   
171   99835971874021   [99820102455001]                1  Original   
176   99925198746002  [999799512733001]                1  Original   
180      30432147001    [1035912594001]                1  Original   
183  999842526707003  [998835355156001]                1  Original   
187     294614647001     [294614640001]                1  Original   
199     122625707001      [46662097001]                1  Original   
207     524356474001     [588183069001]                1  Original   
209      47840025003      [38595605001]                1  Original   
218      46177116001     [852143563001]                1  Original   
223   99823471879001   [99823477824001]                1  Original   
239  998668779648022  [998668779648030]                1  Original   
243     294035978001     [270749727001]                1  Original   
249     684575292001     [684575293001]                1  Original   
254     237035716001      [39331884002]                1  Original   
255    9985711931005    [9985711931007]                1  Original   
256     198924632001     [544616717001]                1  Original   

    conn_src_array  volume  total_conn_vol conn_vol_array conn_tag iteration  \
1       [Original]    1496            1182         [1182]   single  Original   
3       [Original]      49              16           [16]   single  O

In [17]:
# # Insert Data Into DB
# encoded_password = urllib.parse.quote(password)

# connection_string = "postgresql://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
# engine = create_engine(connection_string)

# augmented_df.to_sql("adjacent_geh_script",engine,if_exists='replace',schema=schema_name, chunksize=10000,index=False,method='multi')

258